# Ranking for Maximum Impact

We have a list of charities, $x_1,\ldots,x_n$, and a list of expert judges $e_1,\ldots,e_k$. Each expert judges some charities and gives them a numerical score $e_j(x_i)$. The scores are based on that expert's subjective judgement across a dozen or so criteria, each are scored with a number 1-5 and then are weightedly summed. 

Our question is: how do we rank the charities? 

## model

Let's assume that each charity $x_i$ has a "true" value of $y_i$, which the experts are trying to estimate. 

The experts are additively and multiplicatively biased, and their scores are noisy. We model this as follows:

$$ e_j(x_i) = m_j y_i + b_j + \varepsilon_{j,i}$$

where $m_j$ is the constant multiplicative bias for each expert $e_j$, $b_j$ is the constant additive bias, and $\varepsilon_{j,i} \sim \text{N}(0,\sigma_j)$ is the noise. All variables are assumed to be independent.

## optimization

We are looking for a solution that'd be the "best" fit. A solution would include all of the $y_i$'s, $m_j$'s, $b_j$'s, and $\sigma_j$'s. Note that we have a couple of degrees of freedom here, which amount to changing all $y_i$ by a single linear transformation. We could solve that by normalizing the $y_i$ to have mean 0 and variance 1, for example.

One option would be to define the best fit as the solution that maximizes the likelihood of the available estimates $e_j(x_i)$. After taking logarithm and dropping the constant terms, we get the following expression we want to maximize:

$$ \sum_{i,j} \left( -\frac{1}{2} \log(2\pi\sigma_j^2) - \frac{1}{2\sigma_j^2} (e_j(x_i) - m_j y_i - b_j)^2 \right) $$

where the sum is over all charity/expert pair that's available. Equivalently, we can minimize

$$ 2\sum_j \log(\sigma_j) + \sum_{i,j} \frac{1}{2\sigma_j^2} (e_j(x_i) - m_j y_i - b_j)^2 $$

where the multiplier 2 is the number of charities each expert has (in our case).

This is nearly a convex optimization problem, so we can use a standard solver to find the solution.


[$m_jy_i$ isn't convex. Also $1/\sigma_j^2$ isn't convex]

For simplicity, let's assume that all experts have the same error distribution, i.e. $\sigma_j = \sigma$ for all $j$. Thus, we get a least squares problem:

$$ \min_{y,m,b} \sum_{i,j} (e_j(x_i) - m_j y_i - b_j)^2 $$

In experiments below, this has problems converging. To solve this, we will estimate $m_j$ in two ways. First we will take it to be 1, and second we will estimate it from the data directly for each expert to normalize their own grades. We expect the truth to be somewhere in between, so we'll see how that affects the results.

We also add a regularization term to make sure $b_j$ is small and we normalize the $e_j(x_i)$'s to have mean 0 and variance 1.

Now, we can represent the problem as a matrix equation:

$$ \min_{y,b} \| E - A(y,b) \|_F^2 + \lambda \| b \|_2^2 $$

where $E$ is the matrix of expert scores, $A$ is a $nk,n+k$ matrix taking the concatenation of the vectors $y$ and $b$ to the estimate at each index, and $\lambda$ is the regularization parameter. The matrix $A$ is defined as follows:

$$ A_{(i,j),l} = \begin{cases} \delta_{i,l}m_j & \text{if } l \leq n \\ \delta_{j,l} & \text{otherwise} \end{cases} $$

where $\delta$ is the Kronecker delta (1 if the indices are equal, 0 otherwise). We can then add extra rows to represent the regularization.

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('./df_grades - Sheet1.csv')
df['norm_grade'] = (df.grade - df.grade.mean()) / df.grade.std()
df['m'] = 1
charities = sorted(df['org'].unique())
experts = sorted(df['judge'].unique())
n = len(charities)
k = len(experts)
df.columns

In [ ]:
def create_matrix():
    a = np.zeros([n*k + k, n+k])
    for row in df.iterrows():
        i = charities.index(row[1]['org'])
        j = experts.index(row[1]['judge'])
        m = row[1]['m']
        a[i*k + j, i] = m
        a[i*k + j, n + j] = 1
    for j in range(k):
        a[n*k + j, n + j] = 1
    return a       

def create_target_vector():
    e = np.zeros(n*k + k)
    
    for row in df.iterrows():
        i = charities.index(row[1]['org'])
        j = experts.index(row[1]['judge'])
        e[i*k + j] = row[1]['norm_grade']

    # the last k values are 0

    return e


In [ ]:
a = create_matrix()
e = create_target_vector()
sol = np.linalg.lstsq(a, e, rcond=None)

In [ ]:
y, b = sol[0][:n], sol[0][n:]
sorted(list(zip(charities, y)), key=lambda x: x[1], reverse=True)

In [ ]:
sorted(list(zip(experts, b)), key=lambda x: x[1], reverse=True)

In [ ]:
np.linalg.norm(np.dot(a, sol[0]) - e)

Now, lets test this with $m_j$ calculated to normalize each judge's scores.

In [ ]:
# change df[m] to be the multiplicative normalization factor for each judge's grades

df['m'] = 1
for judge in experts:
    df.loc[df['judge'] == judge, 'm'] = df.loc[df['judge'] == judge, 'norm_grade'].std()

df.head()
    

In [ ]:
a = create_matrix()
e = create_target_vector()
sol = np.linalg.lstsq(a, e, rcond=None)
print(np.linalg.norm(np.dot(a, sol[0]) - e))

In [ ]:
sorted(list(zip(charities, sol[0][:n])), key=lambda x: x[1], reverse=True)

In [ ]:
sorted(list(zip(experts, sol[0][n:])), key=lambda x: x[1], reverse=True)

In [ ]:
# create a graph, where each node is a charity, and edges are differences in normalized grades per judge

import networkx as nx
import matplotlib.pyplot as plt
import itertools

G = nx.DiGraph()
G.add_nodes_from(charities)

for judge in experts:
    for c1, c2 in itertools.combinations(df.loc[df['judge'] == judge, 'org'], 2):
        grade2 = df.loc[(df['judge'] == judge) & (df['org'] == c2), 'norm_grade'].values[0]
        grade1 = df.loc[(df['judge'] == judge) & (df['org'] == c1), 'norm_grade'].values[0]
        if grade1 > grade2:
            G.add_edge(c2, c1, weight=grade1 - grade2)
        else:
            G.add_edge(c1, c2, weight=grade2 - grade1)

def location(node, G=G):
    index = list(nx.topological_sort(G)).index(node)
    # return the location for that index on a circle
    return (np.round(100 * np.cos(2*np.pi*index/n)), np.round(100 * np.sin(2*np.pi*index/n)))

# nx.draw(G, with_labels=True, pos={node: location(node) for node in G.nodes()}, font_size=8)



In [ ]:
# print all cycles, with their weights
cycles = sorted(nx.simple_cycles(G), key=lambda cycle: sum([G[cycle[i % len(cycle)]][cycle[(i+1) % len(cycle)]]['weight'] for i in range(len(cycle))]), reverse=True)
for cycle in cycles:
    print(cycle, sum([G[cycle[i % len(cycle)]][cycle[(i+1) % len(cycle)]]['weight'] for i in range(len(cycle))]))

In [ ]:
len(list(nx.recursive_simple_cycles(G)))

In [ ]:
df.loc[df['judge'] == judge, 'org']


In [ ]:
df.loc[(df['judge'] == judge) & (df['org'] == c1), 'norm_grade']

In [ ]:
# position the charities according to their normalized grades

pos = {node: (200 * df.loc[df['org'] == node, 'norm_grade'].mean() + 500, ((np.pi * sum(ord(c) for c in node)) % 1)*500 ) for node in G.nodes()}

sorted_charities = sorted(charities, key=lambda charity: df.loc[df['org'] == charity, 'norm_grade'].mean(), reverse=True)

nx.draw(G.subgraph(nodes=sorted_charities), pos=pos, font_size=5)
nx.draw_networkx_labels(G.subgraph(nodes=sorted_charities), pos=pos, font_size=5)

In [ ]:
sol[0][:n]

In [ ]:
# position the charities according to their normalized grades
sorted(list(zip(charities, sol[0][:n])), key=lambda x: x[1], reverse=True)

pos = {node: (500 * sol[0][charities.index(node)] + 500, ((1000000* np.pi * sum(ord(c) for c in node)) % 1)*500 ) for node in G.nodes()}

sorted_charities = sorted(charities, key=lambda charity: sol[0][charities.index(charity)], reverse=True)[:10]

nx.draw(G.subgraph(nodes=sorted_charities), pos=pos, font_size=5)
# draw the following nodes in red
selected_nodes = ['Kav LaOved', 'MAF', 'NALA', 'Smoke free Israel', 'Isreali energy forum']
nx.draw_networkx_nodes(G.subgraph(nodes=selected_nodes), pos=pos, node_color='r', node_size=100)
nx.draw_networkx_labels(G.subgraph(nodes=sorted_charities), pos=pos, font_size=5)

In [ ]:
# draw the full graph, sorted by sorted_charities, removing all edges in the direction that agrees with the sorting
sorted_charities = sorted(charities, key=lambda charity: sol[0][charities.index(charity)], reverse=True)
G2 = nx.DiGraph()
G2.add_nodes_from(sorted_charities)
for c1, c2 in itertools.combinations(sorted_charities, 2):
    if (c1, c2) in G.edges():
        G2.add_edge(c1, c2, weight=G[c1][c2]['weight'])


nx.draw(G2, with_labels=True, pos={node: location(node, G2) for node in G2.nodes()}, font_size=8)        


In [ ]:
('20-80', 'NALA') in G.edges()

In [ ]:
for c1 in sorted_charities:
    if (c1, 'Tevel bzedek') in G.edges():
        print(c1)

In [ ]:
for c1 in sorted_charities:
    if ('Tevel bzedek', c1) in G.edges():
        print(c1)

In [ ]:
for c1 in sorted_charities:
    if (c1, 'Robin Food') in G.edges():
        print(c1)

print('------------')

for c1 in sorted_charities:
    if ('Robin Food', c1) in G.edges():
        print(c1)